In [123]:
import requests
import sys
import getpass
from deepdiff import DeepDiff
from pprint import pprint
import json

def compare_dicts(d1, d2):
    diff = DeepDiff(d1,d2, ignore_order=True)
    del diff['values_changed']
    pprint(diff)

def compare_flat_dicts(d1, d2):
    diff = DeepDiff(d1,d2, ignore_order=True)
    del diff['values_changed']
    for diff_key in diff:
        print(diff_key)

        for key in diff[diff_key]:
            value1 = value2 = ""
            key = re.findall(r"root\['([\S\s]+\])'\]", key)[0]
            if key in d1: value1 = d1[key]
            if key in d2: value2 = d2[key]
            print("{}: {} <-> {}".format(key, value1, value2))

def get_string_values_recursive(o, path=[], res={}, ignore_pattern=[]):
    if isinstance(o,dict): 
        #print("dict")
        for k in o:
            _path = path.copy()
            _path.append(k)
            get_string_values_recursive(o[k], _path, res, ignore_pattern)
    elif isinstance(o,list):
        #print("list")
        for i, v in enumerate(o):
            _path = path.copy()
            _path.append(i)
            get_string_values_recursive(o[i], _path, res, ignore_pattern)
    elif isinstance(o,str):
        #print("string")
        path_str = "";
        for p in path: 
            if isinstance(p, str): p = "'" + str(p) + "'"
            path_str += "[" + str(p) + "]"
        ignore = False
        for regex in ignore_pattern:
            match = re.search(regex, path_str)
            if (match != None): ignore = True
        if not ignore: res[path_str] = o;
    return res

def get_flat_dict_total_string_len(d):
    total_chars = 0
    for k in d:
        if isinstance(d[k], str): total_chars += len(d[k])
    return total_chars

def translate(api_key, target_lang, text, source_lang=""):
    text = text.replace('#', "<HASH>") #illegal char for api
    url = 'https://api-free.deepl.com/v2/translate?auth_key={}&text={}&target_lang={}'.format(api_key, text, target_lang); 
    if (source_lang != ""): url += ("&source_lang={}".format(source_lang))
    response = requests.get(url).json()
    if 'translations' in response: response['translations'][0]['text'] = response['translations'][0]['text'].replace("<HASH>", '#')
    return response

def write_dict_to_file(path, d):
    with open(path, 'w', encoding='utf8') as fp:
        json.dump(d, fp, indent=4, ensure_ascii=False)
        
def load_dict_from_file(path):
    with open(path, 'r') as fp:
        d = json.load(fp)
    return d


In [ ]:
api_key = getpass.getpass("Enter DeepL api key")

In [133]:
text = "The `#N/A` error.".replace('#', "<HASH>")
target_lang = 'DE'
url = 'https://api-free.deepl.com/v2/translate?auth_key={}&text={}&target_lang={}'.format(api_key, text, target_lang); 
response = requests.get(url) 
print(response.json());

{'translations': [{'detected_source_language': 'EN', 'text': 'Der `<HASH>N/A`-Fehler.'}]}


In [265]:
en_dict = read_dict_from_file("targets/luckysheet/en.json")
zh_dict = read_dict_from_file("targets/luckysheet/zh.json")
        
d = {'key': 'value', 'key2': [0, 'test'], 'key3': {'k4': 'v4'}}
ignore_regex = []
ignore_regex.append(r"\['functionlist'\][\S\s]*\['n'\]") 
ignore_regex.append(r"\['functionlist'\][\S\s]*\['example'\]")
ignore_regex.append(r"\['functionlist'\][\S\s]*['p'\][\S\s]*\['repeat'\]") 
ignore_regex.append(r"\['functionlist'\][\S\s]*['p'\][\S\s]*\['require'\]") 
ignore_regex.append(r"\['functionlist'\][\S\s]*['p'\][\S\s]*\['name'\]") 
ignore_regex.append(r"\['functionlist'\][\S\s]*['p'\][\S\s]*\['type'\]") 
ignore_regex.append(r"\['dateFmtList'\][\S\s]*") 
ignore_regex.append(r"\['defaultFmt'\][\S\s]*\['value'\]") 
ignore_regex.append(r"\['defaultFmt'\][\S\s]*\['example'\]") 
ignore_regex.append(r"\['drag'\]\['pasteMustKeybordAlertHTML'\]") 

zh_flat = get_string_values_recursive(zh_dict, ignore_pattern=ignore_regex)
en_flat = get_string_values_recursive(en_dict, ignore_pattern=ignore_regex)

#pprint(res)
print(get_flat_dict_total_string_len(en_flat))

124091


In [260]:
124091

In [125]:
#machine translate
target_lang = 'DE'
de_translation = {}
limit = 99999999
counter = 0
for key in en_flat:
    text = en_flat[key]
    response = translate(api_key, 'DE', text)
    de_translation[key] = response
    counter += 1
    print("{}: {}".format(counter, response));
    if counter >= limit: break
#print(de_translation)
write_dict_to_file("targets/luckysheet/en_de_translation.json", de_translation)

{'translations': [{'detected_source_language': 'EN', 'text': 'Gibt eine bedingte Summe über einen Bereich zurück.'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Eine bedingte Summe über einen Bereich.'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Der Bereich, der anhand des Kriteriums geprüft wird.'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Das Muster oder der Test, der auf "range" anzuwenden ist.'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Der zu summierende Bereich, falls abweichend von "Bereich".'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Gibt den Tangens eines Winkels im Bogenmaß zurück.'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Tangens eines Winkels, angegeben im Bogenmaß.'}]}
{'translations': [{'detected_source_language': 'EN', 'text': 'Der Winkel, dessen Tangens zu finden ist, in Radiant.'}]}
{'translations': [{'detected_source_language': 'EN', 'text

In [165]:
de_translation = read_dict_from_file("targets/luckysheet/en_de_translation.json")
for key in de_translation:
    if 'translations' in de_translation[key]:
        if de_translation[key]['translations'][0]['detected_source_language'] != 'EN':
            print("{}={}@{}: {}".format(key, en_flat[key], de_translation[key]['translations'][0]['detected_source_language'], de_translation[key]['translations'][0]['text']))
            if de_translation[key]['translations'][0]['detected_source_language'] != 'ZH':
                text = en_flat[key]
                response = translate(api_key, 'DE', text, source_lang='EN')
                print(response)
                de_translation[key] = response
    if 'message' in de_translation[key]:
        print("{}={}: {}".format(key, en_flat[key], de_translation[key]['message']))
        text = en_flat[key]
        response = translate(api_key, 'DE', text)
        print(response)
        de_translation[key] = response

['functionlist'][248]['a']=Equal.@PT: Gleichberechtigt.
{'translations': [{'detected_source_language': 'EN', 'text': 'Gleichberechtigt.'}]}
['functionlist'][337]['p'][1]['detail']=Date type, default:0:[1900/01/01], 1:[1900-01-01], 2:[1900年1月1日]@ZH: Datumstyp, Standard: 0:[1900/01/01], 1:[1900-01-01], 2:[1. Januar 1900]
['toolbar']['undo']=Undo@IT: Rückgängig machen
{'translations': [{'detected_source_language': 'EN', 'text': 'Rückgängig machen'}]}
['toolbar']['redo']=Redo@DE: Redo
{'translations': [{'detected_source_language': 'EN', 'text': 'Redo'}]}
['toolbar']['font']=Font@PT: Schriftart
{'translations': [{'detected_source_language': 'EN', 'text': 'Schriftart'}]}
['toolbar']['underline']=Underline@DE: Underline
{'translations': [{'detected_source_language': 'EN', 'text': 'Unterstreichen Sie'}]}
['toolbar']['resetColor']=Reset@DE: Reset
{'translations': [{'detected_source_language': 'EN', 'text': 'Zurücksetzen'}]}
['toolbar']['cancelColor']=Cancel@PT: Abbrechen
{'translations': [{'det

In [247]:
#Manual fixes
de_translation["['functionlist'][248]['a']"]['translations'][0]['text'] = 'Gleich.'
de_translation["['toolbar']['redo']"]['translations'][0]['text'] = 'Wiederholen'
de_translation["['toolbar']['autoSum']"]['translations'][0]['text']= 'Auto Summe'
de_translation["['button']['insert']"]['translations'][0]['text']= 'Einfügen'
de_translation["['border']['borderBottom']"]['translations'][0]['text']='Unterer Rand'
de_translation["['merge']['mergeCancel']"]['translations'][0]['text']='Zusammenführung aufheben'
de_translation["['sort']['confirm']"]['translations'][0]['text']='Sortieren'
de_translation["['rightclick']['insert']"]['translations'][0]['text']='Einfügen'
de_translation["['rightclick']['flip']"]['translations'][0]['text']='Wenden'
de_translation["['comment']['insert']"]['translations'][0]['text']='Einfügen'
de_translation["['sheetconfig']['unhide']"]['translations'][0]['text']='Einblenden'
de_translation["['conditionformat']['conditionformat_lessThan']"]['translations'][0]['text']='Bedingungsformat-WenigerAls'
de_translation["['conditionformat']['conditionformat_last10']"]['translations'][0]['text']='Bedingungsformat-Letzte10'
de_translation["['conditionformat']['fillType']"]['translations'][0]['text']='Füllen'
de_translation["['conditionformat']['mark']"]['translations'][0]['text']='Markieren'
de_translation["['formulaMore']['Parser']"]['translations'][0]['text']='Interpreter'
de_translation["['formulaMore']['Operator']"]['translations'][0]['text']='Operator'
de_translation["['functionlist'][115]['d']"]['translations'][0]['text']='Berechnet die Wahrscheinlichkeit für eine links-seitige studentische t-Verteilung mit einer gegebenen Eingabe (x).'
de_translation["['functionlist'][115]['a']"]['translations'][0]['text']='Die links-seitige studentische t-Verteilung mit einer gegebenen Eingabe (x).'
de_translation["['functionlist'][116]['d']"]['translations'][0]['text']='Berechnet die Wahrscheinlichkeit für eine beiseitige studentische t-Verteilung mit einer gegebenen Eingabe (x).'
de_translation["['functionlist'][116]['a']"]['translations'][0]['text']='Die beiseitige studentische t-Verteilung mit einer gegebenen Eingabe (x).'
de_translation["['functionlist'][117]['d']"]['translations'][0]['text']='Berechnet die Wahrscheinlichkeit für eine rechts-seitige studentische t-Verteilung mit einer gegebenen Eingabe (x).'
de_translation["['functionlist'][117]['a']"]['translations'][0]['text']='Die rechts-seitige studentische t-Verteilung mit einer gegebenen Eingabe (x).'
de_translation["['functionlist'][120]['d']"]['translations'][0]['text']='t-Test. Gibt die mit dem studentischen t-test verbundene Wahrscheinlichkeit zurück. Bestimmt, ob zwei Stichproben wahrscheinlich aus denselben zwei zugrunde liegenden Populationen stammen, die denselben Mittelwert haben.'
de_translation["['functionlist'][122]['d']"]['translations'][0]['text']='Berechnet die rechts-seitige F-Wahrscheinlichkeitsverteilung (Grad der Diversität) für zwei Datensätze mit gegebener Eingabe x. Auch Fisher-Snedecor-Verteilung oder Snedecors F-Verteilung genannt.'
de_translation["['functionlist'][230]['p'][2]['detail']"]['translations'][0]['text']='Das Suffix für den Imaginärkoeffizienten kann nur `i` oder `j` sein. Wird er weggelassen, wird `i` verwendet.'
de_translation["['currencyDetail']['TonganPaanga']"]['translations'][0]['text']='tongaischer Pa`anga'
de_translation["['sheetconfig']['sheetNameSpecCharError']"]['translations'][0]['text']="Der Name darf nicht enthalten:[ ] : ? * / ' \""

In [261]:
#copy translations into en.json structure
de_flat = {}
for key in de_translation:
    de_flat[key] = de_translation[key]['translations'][0]['text']

de_dict = load_dict_from_file('targets/luckysheet/en.json')
for key in de_flat:
    #print(key)
    #exec("de_dict{} = r\"\"\"{}\"\"\"".format(key, de_flat[key]))#.replace('\n','\\n')))#.replace("'","\'")))
    if '"' in de_flat[key]: exec("de_dict{} = '{}'".format(key, de_flat[key].replace("\'","\\'")))
    else: exec('de_dict{} = "{}"'.format(key, de_flat[key].replace('\n','\\n')))
write_dict_to_file('targets/luckysheet/de.json', de_dict)

In [255]:
de_flat["['functionlist'][61]['p'][4]['detail']"]
de_dict['functionlist'][61]['p'][4]['detail']
de_flat["['sheetconfig']['sheetNameSpecCharError']"]
de_dict['sheetconfig']['sheetNameSpecCharError']
#de_flat["['functionlist'][120]['d']"]

'Der Name darf nicht enthalten:[ ] : ? * / \' "'